In [82]:
key = "tI6XMGQoGKaxRK7CFXSij99gB3FwfuBT674ZLt2GzzDkG4eq8Q"
secret = "iEVGwWWl6z0gextpt98ktTsg6hWEUXoe8LieIsmi"

In [2]:
import petpy

In [83]:
pf = petpy.Petfinder(key=key, secret=secret)

In [8]:
animal_types = pf.animal_types()

In [10]:
animal_breeds = pf.breeds()

for breed in animal_breeds['breeds']:
    print(breed)
    print(animal_breeds['breeds'][breed])

dog
['Affenpinscher', 'Afghan Hound', 'Airedale Terrier', 'Akbash', 'Akita', 'Alaskan Malamute', 'American Bulldog', 'American Bully', 'American Eskimo Dog', 'American Foxhound', 'American Hairless Terrier', 'American Staffordshire Terrier', 'American Water Spaniel', 'Anatolian Shepherd', 'Appenzell Mountain Dog', 'Aussiedoodle', 'Australian Cattle Dog / Blue Heeler', 'Australian Kelpie', 'Australian Shepherd', 'Australian Terrier', 'Basenji', 'Basset Hound', 'Beagle', 'Bearded Collie', 'Beauceron', 'Bedlington Terrier', 'Belgian Shepherd / Laekenois', 'Belgian Shepherd / Malinois', 'Belgian Shepherd / Sheepdog', 'Belgian Shepherd / Tervuren', 'Bernedoodle', 'Bernese Mountain Dog', 'Bichon Frise', 'Black and Tan Coonhound', 'Black Labrador Retriever', 'Black Mouth Cur', 'Black Russian Terrier', 'Bloodhound', 'Blue Lacy', 'Bluetick Coonhound', 'Boerboel', 'Bolognese', 'Border Collie', 'Border Terrier', 'Borzoi', 'Boston Terrier', 'Bouvier des Flandres', 'Boxer', 'Boykin Spaniel', 'Briar

In [4]:
CA_shelter = pf.organizations(pages=None, state='CA')

In [5]:
import json

with open('ca_shelter_0112.json', 'w') as json_file:
    json.dump(CA_shelter, json_file)

In [6]:
with open('ca_shelter_0112.json') as f:
    ca_shelter = json.load(f)

In [33]:
#Fetch all animals by organization id
id_list = []
for shelter in ca_shelter['organizations']:
    shelter_id = shelter['id']
    if shelter_id not in id_list:
        id_list.append(shelter_id)

In [55]:
id_test = id_list[:2]
id_test

['CA2464', 'CA458']

In [60]:
ca_animals = pf.animals(pages=None, organization=['CA458','CA2464'])
##error by quering id_List

TypeError: animals() got an unexpected keyword argument 'organization'

In [59]:
for i in ca_animals['animals']:
    print(i['organization_id'])

CA2464


In [7]:
#Fetch by city,CA
address_list = []
for shelter in ca_shelter['organizations']:
    shelter_address = shelter['address']['city'].strip().lower().title() + ', CA'
    if shelter_address not in address_list:
        address_list.append(shelter_address)

In [8]:
len(address_list)
#only 443 cities in CA, so I can use for loop to run the API calls

419

In [ ]:
animal_list = []
animal_id = []
for a in address_list:
    try:
        city_animals = pf.animals(location = a, pages=None)
    except petpy.exceptions.PetfinderInvalidParameters:
        print(f"Error querrying {a}")
        continue
    except Exception as x:
        print(x)
    else:
        for animal in city_animals['animals']:
            if animal['id'] not in animal_id:
                animal_list.append(animal)
                animal_id.append(animal['id'])

## With location specified, there is a default distance to be 50 miles for the search.
## So there are a lot lot lot of duplicate results while sampling different cities becuase
## of the range. A lot of cities are within 50 miles range. And I cannot set a definitive
## limit for that due to uneven distribution of city locations.

In [ ]:
## Instead, the dumpest way is to specify a lat-long location and set the distance
## to maximum 500 miles that inclues the whole california. Of cause then there will be 
## results outside CA. We'll filter then out later. 

In [84]:
animal_list = []

try:
    animals_all = pf.animals(location = '35.466895,-122.115810', distance=500, pages=None)
except Exception as x:
    print(x)
else:
    for animal in animals_all['animals']:
        if animal['contact']['address']['state'] == "CA" and (animal not in animal_list):
            animal_list.append(animal)

In [85]:
ca_animals = {'animals': animal_list}

In [86]:
len(ca_animals['animals'])

12592

In [87]:
with open('ca_data_0122.json', 'w') as json_file:
    json.dump(ca_animals, json_file)

In [88]:
## check unique ID 
animal_check = []
duplicate_id = []
for animal in ca_animals['animals']:
    if animal not in animal_check:
        animal_check.append(animal)
        if animal['id'] not in duplicate_id:
            duplicate_id.append(animal['id'])
        else:
            print(f"There is a duplicate for {animal['id']}, but they are different")
    else:
        print(f"There is a duplicate for {animal['id']}, but they are the same")

##OKAY looks like the duplicates were generated through API calling. Duplicates are all the
##same, so we can confiently remove duplicate.

TypeError: unhashable type: 'dict'

In [69]:
duplicates

{'animals': {'id': 50173427,
  'organization_id': 'CA2043',
  'url': 'https://www.petfinder.com/cat/jessie-50173427/ca/mission-viejo/compassionate-animal-rescue-for-medical-aid-ca2043/?referrer_id=edf7097c-862d-4871-a9ad-9278bcecc340',
  'type': 'Cat',
  'species': 'Cat',
  'breeds': {'primary': 'Domestic Short Hair',
   'secondary': None,
   'mixed': False,
   'unknown': False},
  'colors': {'primary': 'Black & White / Tuxedo',
   'secondary': None,
   'tertiary': None},
  'age': 'Baby',
  'gender': 'Female',
  'size': 'Small',
  'coat': None,
  'attributes': {'spayed_neutered': True,
   'house_trained': True,
   'declawed': False,
   'special_needs': False,
   'shots_current': True},
  'environment': {'children': None, 'dogs': None, 'cats': True},
  'tags': [],
  'name': 'Jessie',
  'description': 'I am currently available for adoption at the Lake Forest Petsmart. If you&amp;#39;re interested in adopting me, fill out an...',
  'organization_animal_id': '16392144',
  'photos': [{'smal